# Generate CDFG Samples

In [1]:
import os
import yaml
import glob
import numpy as np
import random
import tqdm

from typing import Tuple, List, Dict
from matplotlib import pyplot as plt

from utils import yaml_load, yaml_write, get_section

%matplotlib inline

## Utilitary Functions

These utilitary functions are used to select CDFGs and generate adjacency and features matrices to be used by GNNs.

These functions use the database file to several cases, such as: check the number of nodes in the CDFGs, name of the files from a certian application and otimization sequence, *etc*.. 

The database file is a dictionary that contains information about applications and their optimization sequences, like this:

```python
{
    # Information about application 1
    application_1: {
        # Application 1 optimized with sequence 0
        0: {
            cfg_file: '/path/to/cfg/file'
            number_cfg_nodes: 10
            cdfg_file: '/path/to/cdfg/file'
            number_cdfg_nodes: 20
            exectime: 1.234
        }
        # Application 1 optimized with sequence 1
        1: {   
            cfg_file: '/path/to/cfg/file'
            number_cfg_nodes: 15
            cdfg_file: '/path/to/cdfg/file'
            number_cdfg_nodes: 22
            exectime: 0.987
        }
    }
    
    # Information about application 2
    application_2: {
        # Application 2 optimized with sequence 0
        0: {
            ...
        }
        ...
    }
    ...
 }
```

See `building_metadata` notebook for more information about database file.

In [2]:
def filter_cdfg_files_below_order(metadata: dict, order: int) -> dict:
    """Filter CDFG files from metadata database which adjacency matrix is below some order (number of nodes).
    
    Parameters:
        metadata (dict): Dictionary of metadata datasabe. See `building_metadata` notebook for explanation about metadata
        order (int): Order of adjacency matrix to filter
    
    Returns:
        The filtered metadata database dictionary  
    """
    # Lets filter CDFG files in which number of nodes is less than an order
    filtered_graph_files = {
        benchmark_name: {
            opt_seq: opt_values 
            for opt_seq, opt_values in values.items() if opt_values['number_cdfg_nodes'] < order
        } for benchmark_name, values in metadata.items()
    }

    # Remove applications that has less than 2 graphs
    filtered_graph_files = {benchmark_name: values for benchmark_name, values in filtered_graph_files.items() if len(values) > 2}
    print(f"Number of graphs with adjacency matrix below order {order}: {sum([len(values) for _, values in filtered_graph_files.items()])}")
    # Return filtered metadata
    return filtered_graph_files

def sample_cdfg_files(metadata: dict, num_samples: int, sample_condition: callable = None, weighted: bool = True, tries: int = 10) -> List[Tuple[str, str, float]]:
    """Sample CDFGs from metadata dict which satisfy a sample condition. Each sample contains 2 CDFG files of the same application with different optimization sequences. 

    Parameters:
        metadata (dict): Dictionary of metadata datasabe.
        num_samples (int): Number of samples to pick
        sample_condition (callable): A function determining if the two CDFG files satisfy a condition. The function must receive 2 parameters and return a
                                boolean indicating if the sample is valid (True) or not (False). The parameters are 2 dictionaries with information about the 
                                CDFGs. Each dictionary contains the following information:
                                * cfg_file (str): /path/to/cfg/file
                                * cdfg_file (str): /path/to/cdfg/file
                                * number_cdfg_nodes (int): Number of nodes in CDFG representation
                                * number_cfg_nodes (int): Number of nodes in CFG representation
                                * exectime: Execution time of the application
        weighted (bool): True if selection of samples must by weighted by application and False otherwise
        
    Returns:
        A list of tuples, with `num_samples` elements, where each tuple has the following elements:
        * Path to first CDFG file
        * Path to second CDFG file
        * Relative speedup. Execution time of application from first CDFG over execution time of application from second CDFG
        Returns None if the informed number of samples is not reached
    """
    samples = []
    apps = list(metadata.keys())
    
    if weighted:
        # Weighted selection based on how many sequeces is in each app
        total_app_seq = sum([len(opt_sequences) for _, opt_sequences in metadata.items()])
        weights = [len(opt_sequences)/total_app_seq for _, opt_sequences in metadata.items()]
    else:
        # Equal weights for all elements
        weights = [1/len(apps)] * len(apps)
    
    # Returns None if no sample that satisfy condition match
    def try_sample(tries: int = 10):
        """Inner function used to pick a sample (i.e., two graph files app:opt1, app:opt2) that satisfy a sample condition

        Params:
            tries (int): NUmber of tries to pick a sample that satisfy condition.

        Returns:
            A 3-element tuple with:
            * The name of the application
            * The 1st optimization sequence for that application
            * The 2nd optimization sequence for that application
            Returns None if could not pick any sample that matches the sample condition 
        """
        # Tries to pick two CDFGs for same application for 'tries' times
        for i in range(0, tries):
            # Choose an application
            app = np.random.choice(apps, p=weights)
            # Choose 2 optimization sequences for that application
            opt_seq1, opt_seq2 = random.sample(list(metadata[app].keys()), 2)
            # Does it have a sampling condition?
            if sample_condition is not None:
                # Applies the condition
                if sample_condition(metadata[app][opt_seq1], metadata[app][opt_seq2]) == True:
                    return (app, opt_seq1, opt_seq2)
            # If no sampling condition is given, just return the picked value 
            else:
                return (app, opt_seq1, opt_seq2)
        # Return None if no sample was retrieved
        return None
        
    # Pick 'num_samples' samples
    for i in range(0, num_samples):
        # Try to pick a sample for 'tries' times
        sample = try_sample(tries=tries)
        # Raises exception if no samples could be picked
        if not sample:
            raise Exception("Could not generate number of samples with the valid sample condition")
        
        # Generate a tuple (CDFG1, CDFG2, RelativeSpeedup) and append to samples list
        app, opt_seq1, opt_seq2 = sample
        t = (
            metadata[app][opt_seq1]['cdfg_file'], 
            metadata[app][opt_seq2]['cdfg_file'],
            metadata[app][opt_seq1]['exectime']/metadata[app][opt_seq2]['exectime']            
        )
        samples.append(t)

    # Return samples
    return samples

no_total_features = 0
no_total_invalid_features = 0

def generate_graph_feature_matrices(network_shape: Tuple[int, int], features_shape: Tuple[int, int], edges: dict, edge_features: dict, node_features: dict, representation: tuple, embeddings_dict: dict) -> Tuple[np.array, np.array]:
    """Generate the graph's adjacency matrix and the feature matrix from sparse data (dict).
    
    Parameters:
        network_shape (tuple):  Shape of the graph's adjacency matrix used as input to GNN. 
                                The matrix is filled with zeros if the graph has less nodes than network_shape
        features_shape (tuple): Shape of the feature matrix used as input to GNN
                                The matrix is filled with zeros if the graph has less nodes than features_shape
        edges (dict): Dictionary with the edges. Each key is the edge id and each value is a 2-element tuple with origin and destiny of the edge
        edge_features (dict): Dictionary with the type of each edge. Each key is the edge id and each value is a 1-element tuple with the edge's type. Type is 0, 1 or 2, for control, data and call edges
        node_features (dict): Dictionary describing the feature vector of each node. Each dictionary's key is the node_id and each value is the ID of an inst2vec feature vector. 
        representation (tuple): Lists which representations must be included in the graph. Must be any combination of these values: 'control', 'data' or 'call'.
        embeddings_dict (dict): Embbeding inst2vec dictionary. Each key is the inst2vec feature vector's ID and the each value is a 200-element list describing the feature vector
        
    Returns:
        A 2-element tuple:
        * The graph matrix
        * The feature matrix
    """
    
    global no_total_features, no_total_invalid_features
    # Which edges to include in the graph
    valid_edges = {
        0: 'control' in representation,
        1: 'data' in representation,
        2: 'call' in representation
    }
    
    adj_matrix = np.full((network_shape), False, dtype='bool')
    feature_matrix = np.zeros((features_shape), dtype='float32')
    
    # Fill the graph's adjacency matrix with edges
    for edge_no, edge in edges.items():
        edge_type = edge_features[edge_no][0]
        if valid_edges[edge_type] == True:
            adj_matrix[edge[0]][edge[1]] = True
    
    # non_zero_nodes = {node_no: is_not_zero_row for node_no, is_not_zero_row in enumerate(adj_matrix.any(axis=1))}
    
    # Add inst2vec features to nodes
    for node_no, feature_no in node_features.items():
        # if non_zero_nodes[node_no] == True:
        feature_no = feature_no[0]
        feature_matrix[node_no] = embeddings_dict[feature_no]
        if feature_no == 8564:
            no_total_invalid_features += 1
        no_total_features += 1
    
    return adj_matrix, feature_matrix  

def load_sample_files(sample_tuple: Tuple[str, str, float], network_shape: Tuple[int, int], features_shape: Tuple[int, int], representation: tuple, embeddings_dict: dict) -> Tuple[np.array, np.array, np.array]:
    """Given a tuple with CDFG filenames, load each CDFG and generate their adjacency and feature matrices.
    
    Parameters:
        sample_tuple (tuple): A 3-element tuple, with:
                * Path to first CDFG file
                * Path to second CDFG file
                * Relative speedup. Execution time of application from first CDFG over execution time of application from second CDFG
        network_shape (tuple):  Shape of the graph's adjacency matrix used as input to GNN. 
        features_shape (tuple): Shape of the feature matrix used as input to GNN
        representation (tuple): Lists which representations must be included in the graph. Must be any combination of these values: 'control', 'data' or 'call'.
        embeddings_dict (dict): Embbeding inst2vec dictionary. Each key is the inst2vec feature vector's ID and the each value is a 200-element list describing the feature vector

    Returns:
        A 3-element tuple with:
        * A matrix with 2 CDFGs
        * A matrix with 2 CDFGs features
        * The speedup (1-element matrix)
    """
    
    def load_graph_from_file(filename: str) -> Tuple[dict, dict]:
        """Inner function used to load the graph from a file
        
        Parameters:
            filename (str): Name of the file with CDFG
        
        Returns:
            A 2-element tuple with the graph, represented as adjacency list and the features
        """
        x = yaml_load(filename)
        return x['edges'], x['edges_features'], x['nodes_features']
    
    # Load graph from first file and generate the graph and feature matrices
    edges_1, edge_feat_1, feat_1 = load_graph_from_file(sample_tuple[0])
    graph1, features1 = generate_graph_feature_matrices(
        network_shape, features_shape, edges_1, edge_feat_1, 
        feat_1, representation_to_generate, embeddings_dict)
    # Load graph from second file and generate the graph and feature matrices
    edges_2, edge_feat_2, feat_2 = load_graph_from_file(sample_tuple[1])
    graph2, features2 = generate_graph_feature_matrices(
        network_shape, features_shape, edges_2, edge_feat_2, 
        feat_2, representation_to_generate, embeddings_dict
    )
 
    # Generate a matrix with the 2 graph matrices
    graphs = np.array([graph1, graph2], dtype='bool')
    # Generate a matrix with the 2 feature matrices
    features = np.array([features1, features2], dtype='float32') 
    # Generate 1-element speedup array
    speedup_array = np.array([sample_tuple[2]], dtype='float32')
    
    return graphs, features, speedup_array


def generate_samples(samples: List[Tuple[str, str, float]], network_shape: Tuple[int, int], feature_shape: Tuple[int, int], representation: tuple, embeddings_dict: dict, desc: str = 'Samples generated'):
    """Generate samples used as input to a GNN
    
    Parameters:
        samples (list): List of tuple, where each tuple has the following elements:
                * Path to first CDFG file
                * Path to second CDFG file
                * Relative speedup. Execution time of application from first CDFG over execution time of application from second CDFG
        network_shape (tuple):  Shape of the graph's adjacency matrix used as input to GNN. 
        features_shape (tuple): Shape of the feature matrix used as input to GNN
        representation (tuple): Lists which representations must be included in the graph. Must be any combination of these values: 'control', 'data' or 'call'.
        embeddings_dict (dict): Embbeding inst2vec dictionary. Each key is the inst2vec feature vector's ID and the each value is a 200-element list describing the feature vector
        desc (str): Optional description for TQDM bar
        
    Returns:
        A 3-element tuple with:
        * A matrix with all samples, each sample has 2 CDFGs
        * A matrix with all samples, each sample has 2 CDFGs features
        * A matrix with all samples, each sample has the speedup
    """    
    global no_total_features, no_total_invalid_features

    for rep in representation:
        assert rep in ['control', 'data', 'call'], f"Invalid representation {rep}"
    
    sampled_graphs = np.zeros((len(samples), 2, network_shape[0], network_shape[1]), dtype='bool')
    sampled_features = np.zeros((len(samples), 2, feature_shape[0], feature_shape[1]), dtype='float32')
    sampled_speedups = np.zeros((len(samples), 1), dtype='float32')

    # Some counters
    equal_graphs = 0
    equal_graphs_features = 0
    no_total_features = 0
    no_total_invalid_features = 0
    
    # Load each sample and store in input_graphs, input_features and speedups
    for i in tqdm.tqdm(range(0, len(samples)), desc=desc):
        # This function return a list of tuples, where each tuple is composed by:
        # np.array with 2 graphs, np.array with 2 features, speedup
        graphs, features, speedup = load_sample_file(samples[i], network_shape, feature_shape, representation, embeddings_dict)           
        sampled_graphs[i] = graphs
        sampled_features[i] = features
        sampled_speedups[i] = speedup
 
        if np.array_equal(graphs[0], graphs[1]):
            equal_graphs += 1
            if np.array_equal(features[0], features[1]):
                equal_graphs_features += 1

    print(f"Number of samples loaded: {len(samples)}")
    print(f"Graphs shape: {sampled_graphs.shape}")
    print(f"Features shape: {sampled_features.shape}")
    print(f"Speedups (target) shape: {sampled_speedups.shape}")
    print(f"Number of samples with equal graphs: {equal_graphs}")
    print(f"Number of samples with equal graphs and equal features: {equal_graphs_features}")
        
    print(f"Number of features assigned (FA): {no_total_features}")
    print(f"Invalid number of representations assigned: {no_total_invalid_features} (in relation to FA: {no_total_invalid_features/no_total_features}%)")
    
    return sampled_graphs, sampled_features, sampled_speedups

In [ ]:
def get_output_filenames(data_dir: str, num_samples: int, network_shape: Tuple[int, int], tag) -> Tuple[str, str]:
    """Generate the file paths to store the samples
    
    Parameters:
        data_dir (str): Root data directory to store information
        num_samples (int): Number of samples generated
        network_shape (tuple):  Shape of the graph's adjacency matrix used as input to GNN
        
    Returns:
        A 2-element tuple with:
        * The name of the file to store the samples
        * The name of the file to store samples information (which CDFG files were selected)
    """
    output_data_file = f"cdfgs_{tag}_{num_samples}samples_{network_shape[0]}x{network_shape[1]}"
    output_data_file = os.path.join(data_dir, output_data_file)
    
    selected_data_file = f"selected_cdfgs_{tag}_{num_samples}samples_{network_shape[0]}x{network_shape[1]}.yaml"
    selected_data_file = os.path.join(data_dir, selected_data_file)
    return output_data_file, selected_data_file

def save_data_file(output_data_file: str, output_samples_file: str, samples: List[Tuple[str, str, float]], graphs: np.array, features: np.array, speedups: np.array):
    """Save samples to a .npz file
    
    Parameters:
        output_data_file (str): Path to store the samples to npz file (graphs, feaures and speedups)
        output_samples_file (str): Path to store information about samples (which files correspond to each sample)
        samples (list): Files used to generate the samples
        graphs (np.array): Matrix with all graphs
        features (np.array): Matrix with all features
        speedups (np.array): Matrix with all speedups
        
    Returns:
        None
    """
    np.savez_compressed(output_data_file, graphs=graphs, features=features, speedups=speedups)
    print(f"Data saved to {output_data_file}.npz")
    
    yaml_write(output_samples_file, samples)
    print(f"Samples information saved to {output_samples_file}")

## Common variables for all shapes

In [ ]:
# Default root directory to store all informations 
data_dir = './data'
# Metadata database file
metadata_file = './data/ccpe-applications-information.yaml'
# Load metadata database
metadata_info = yaml_load(metadata_file)
print(f"Metadata loaded from '{metadata_file}'")

# Embeddings file
embeddings_file = './data/inst2vec/emb.p'
# Embeddings dict
embeddings_info = np.load(embeddings_file, allow_pickle=True)
print(f'Embbedings loaded from {embeddings_file}')

## Generating representations for (150x150) shape

CDFG representations with the following edges:
- Control, data and call edges
- Control and data edges

The samples generation is composed by 4 steps:
1. Remove CDFG files where the number of CDFG nodes is higher than a order (speficied by network_graph_shape variable). 
2. Sample CDFG files. This will generate a list of tuples, where each tuple is composed by: path to CDFG1, path to CDFG2, speedup between CDFG1/CDFG2. The two CDFGs are from the same application, but with different optimization sequences.
3. Generate representations from sampled CDFG files (adjacency matrices, feature matrices and speedup matrices)
4. Store samples to .npz file

In [ ]:
# Defining some variables
network_graph_shape = (150, 150)    # Network input graph's shape
network_features_shape = (150, 67)  # Network input feature's shape
n_samples = 20000                   # Number of samples to generate. Each sample is composed by 2 graphs
representations_to_generate = [
    ('control', 'data', 'call'),
    ('control', 'data'),
    #('control', 'call'),
    #('data', 'call'),
    #('control',),
    #('data',),
    #('call',)
]

# 1. Filtering CFGs that number of nodes is below 'network_graph_shape'
cfg_files = filter_cdfg_files_below_order(metadata_info, network_graph_shape[0])

# Conditions to compose each class of samples (A, B, C and D)
classA_cond = lambda opt1, opt2: opt1['exectime']/opt2['exectime'] < 0.45
classB_cond = lambda opt1, opt2: opt1['exectime']/opt2['exectime'] <= 0.80 and opt1['exectime']/opt2['exectime'] >= 0.45
classC_cond = lambda opt1, opt2: opt1['exectime']/opt2['exectime'] <= 1.33 and opt1['exectime']/opt2['exectime'] > 0.80
classD_cond = lambda opt1, opt2: opt1['exectime']/opt2['exectime'] > 1.33

# 2. Sample CFG files based on conditions. 1/4 of CFG files for each condition. 
print("Class A samples...")
sample_files =  sample_cdfg_files(cfg_files, n_samples//4, sample_condition=classA_cond, tries=1000)
print("Class B samples...")
sample_files += sample_cdfg_files(cfg_files, n_samples//4, sample_condition=classB_cond, tries=1000)
print("Class C samples...")
sample_files += sample_cdfg_files(cfg_files, n_samples//4, sample_condition=classC_cond, tries=1000)
print("Class D samples...")
sample_files += sample_cdfg_files(cfg_files, n_samples//4, sample_condition=classD_cond, tries=1000)
    
for representation in representations_to_generate:
    print(f'Generating representation for: {representation}')
    
    output_data_file, output_samples_file = get_output_filenames(data_dir, n_samples, network_graph_shape, '-'.join(representation))  # Paths to store samples
    
    # 3. Generate samples from files
    graphs, features, speedups = generate_samples(sample_files, network_graph_shape, network_features_shape, representation, embeddings_info, desc=f'Samples for representation {representation}')

    # 4. Save all samples to a .npz file
    save_data_file(output_data_file, output_samples_file, sample_files, graphs, features, speedups)
    
    print(f"Finished generation for representation {representation}\n")

## NOt used

In [ ]:
while True:
    print('Not used')
    time.sleep(1)

In [3]:
# Load the graph and features from file
no_total_features = 0
no_total_invalid_features = 0

def load_graph_from_file(filename: str) -> tuple:
    x = yaml_load(filename)
    return x['edges'], x['edges_features'], x['nodes_features']

def generate_graph_matrix(network_shape, features_shape, edges, edge_features, node_features, representation, embeddings_dict: dict) -> tuple:
    global no_total_features, no_total_invalid_features
    valid_edges = {
        0: 'control' in representation,
        1: 'data' in representation,
        2: 'call' in representation
    }
    
    # print(f'Valid edges: {valid_edges}')
    
    adj_matrix = np.full((network_shape), False, dtype='bool')
    feature_matrix = np.zeros((features_shape), dtype='float32')
    
    for edge_no, edge in edges.items():
        edge_type = edge_features[edge_no][0]
        if valid_edges[edge_type] == True:
            adj_matrix[edge[0]][edge[1]] = True
    
    non_zero_nodes = {node_no: is_not_zero_row for node_no, is_not_zero_row in enumerate(adj_matrix.any(axis=1))}
    
    for node_no, feature_no in node_features.items():
        if non_zero_nodes[node_no] == True:
            feature_no = feature_no[0]
            feature_matrix[node_no] = embeddings_dict[feature_no]
            if feature_no == 8564:
                no_total_invalid_features += 1
        no_total_features += 1
        
    # print(adj_matrix)
    # print(feature_matrix)
    return adj_matrix, feature_matrix  
    

def load_sample_file(sample_tuple: tuple, network_shape: tuple, features_shape: tuple, representation_to_generate: tuple, embeddings_dict: dict) -> tuple:
    # print(f'Loading first file: {sample_tuple[0]}')
    edges_1, edge_feat_1, feat_1 = load_graph_from_file(sample_tuple[0])
    # print(f'Loading second file: {sample_tuple[1]}')
    edges_2, edge_feat_2, feat_2 = load_graph_from_file(sample_tuple[1])
    
    graph1, features1 = generate_graph_matrix(
        network_shape, features_shape, edges_1, edge_feat_1, 
        feat_1, representation_to_generate, embeddings_dict)
    graph2, features2 = generate_graph_matrix(
        network_shape, features_shape, edges_2, edge_feat_2, 
        feat_2, representation_to_generate, embeddings_dict)
    
    graphs = np.zeros((2, network_shape[0], network_shape[1]), dtype='bool')
    graphs[0] = graph1
    graphs[1] = graph2

    features = np.zeros((2, features_shape[0], features_shape[1]), dtype='float32')
    features[0] = features1
    features[1] = features2
    
    speedup_array = np.array([sample_tuple[2]], dtype='float32')

    # Returns:
    # * A matrix with 2 graphs with shape: (2, network_input_graph_shape[0], network_input_graph_shape[1])
    # * A matrix with 2 features with shape: (2, network_input_features_shape[0], network_input_features_shape[1])
    # * A float with speedup of CFG1/CFG2
    return graphs, features, speedup_array


def generate_samples(samples: list, network_shape: tuple, feature_shape: tuple, representation: tuple, embeddings_dict: dict, desc: str = 'Samples generated'):
    global no_total_features, no_total_invalid_features
    no_total_features = 0
    no_total_invalid_features = 0
    for rep in representation:
        assert rep in ['control', 'data', 'call'], f"Invalid representation {rep}"
    
    input_graphs = np.empty((len(samples), 2, network_shape[0], network_shape[1]), dtype='bool')
    input_features = np.empty((len(samples), 2, feature_shape[0], feature_shape[1]), dtype='float32')
    speedups = np.empty((len(samples), 1), dtype='float32')

    # Some counters
    i = 0
    equal_graphs = 0
    
    # Load each sample and store in input_graphs, input_features and speedups
    for sample in tqdm.tqdm(samples, desc=desc):
        # This function return a list of tuples, where each tuple is composed by:
        # np.array with 2 graphs, np.array with 2 features, speedup
        graphs, features, speedup = load_sample_file(sample, network_shape, feature_shape, representation, embeddings_dict)           
        input_graphs[i] = graphs
        input_features[i] = features
        speedups[i] = speedup
        i += 1
        if np.array_equal(graphs[0], graphs[1]):
            equal_graphs += 1

    print(f"Number of samples loaded: {len(samples)}")
    print(f"Graphs shape: {input_graphs.shape}")
    print(f"Features shape: {input_features.shape}")
    print(f"Speedups (target) shape: {speedups.shape}")
    print(f"Number of samples with equal graphs: {equal_graphs}")
    
    print(f"Number of features assigned (FA): {no_total_features}")
    print(f"Invalid number of representations assigned: {no_total_invalid_features} (in relation to FA: {no_total_invalid_features/no_total_features}%)")
    
    #np.savez_compressed(output_file, graphs=input_graphs, features=input_features, speedups=speedups)
    #print(f"Representation {representation} saved to {output_file}")
    return input_graphs, input_features, speedups

# Common variables for all representations

In [4]:
data_dir = './data'
metadata_file = './data/ccpe-applications-information.yaml'
metadata_info = yaml_load(metadata_file)
print('Metadata loaded')

# Embeddings
embeddings_file = './data/inst2vec/emb.p'
embeddings_info = np.load(embeddings_file, allow_pickle=True)
print('Embbedings loaded')

Metadata loaded
Embbedings loaded


In [5]:
def get_output_filenames(data_dir: str, representation: tuple, num_samples, network_shape) -> tuple:
    output_data_file = f"cdfgs_{'-'.join(representation)}_{num_samples}samples_{network_shape[0]}x{network_shape[1]}"
    output_data_file = os.path.join(data_dir, output_data_file)
    
    selected_data_file = f"selected_cdfgs_{'-'.join(representation)}_{num_samples}samples_{network_shape[0]}x{network_shape[1]}.yaml"
    selected_data_file = os.path.join(data_dir, selected_data_file)
    return output_data_file, selected_data_file

In [6]:
def save_data_file(output_data_file: str, output_samples_file: str, samples: list, graphs, features, speedups):
    np.savez_compressed(output_data_file, graphs=graphs, features=features, speedups=speedups)
    print(f"Data saved to {output_data_file}.npz")
    
    with open(output_samples_file, 'wt') as f:
        yaml.dump(samples, f)
    print(f"Samples information saved to {output_samples_file}")

# Generate representations for (150x150) shape

### Common variables for all 150x150 representations

In [7]:
# Defining some useful variables
network_graph_shape = (150, 150)
network_features_shape = (150, 200)
n_samples = 20000  # Number of samples to generate. Each sample is composed by 2 graphs

cdfg_files = filter_cdfgs_files_below_shape(metadata_info, network_graph_shape)
samples = sample_cdfgs(cdfg_files, n_samples)

Number of graphs with adjacency matrix below shape (150, 150): 887
Sampled 20000 CDFGs
First sample of 20000 samples (example of output): ('./data/ccpe-dados/cdfg.programl/programl.5000.01/extracted/BenchmarkGame.nsieve-bits.0.11.progaml.yaml', './data/ccpe-dados/cdfg.programl/programl.5000.01/extracted/BenchmarkGame.nsieve-bits.0.2.progaml.yaml', 1.0059670138388164)


### Data generation

In [8]:
representations_to_generate = [
    ('control', 'data', 'call'),
    ('control', 'data'),
    ('control', 'call'),
    ('data', 'call'),
    ('control',),
    ('data',),
    ('call',)
]

for representation in representations_to_generate:
    print(f'Generating representation for: {representation}')
    output_data_file, output_samples_file = get_output_filenames(data_dir, representation, n_samples, network_graph_shape)
    graphs, features, speedups = generate_samples(samples, network_graph_shape, network_features_shape, representation, embeddings_info)
    save_data_file(output_data_file, output_samples_file, samples, graphs, features, speedups)
    print('Finished generation\n')

Samples generated:   0%|          | 7/20000 [00:00<05:07, 64.95it/s]

Generating representation for: ('control', 'data', 'call')


Samples generated: 100%|██████████| 20000/20000 [07:04<00:00, 47.12it/s]


Number of samples loaded: 20000
Graphs shape: (20000, 2, 150, 150)
Features shape: (20000, 2, 150, 200)
Speedups (target) shape: (20000, 1)
Number of samples with equal graphs: 0
Number of features assigned (FA): 4709769
Invalid number of representations assigned: 3086242 (in relation to FA: 0.6552852167484222%)
Data saved to ./data/cdfgs_control-data-call_20000samples_150x150.npz


Samples generated:   0%|          | 4/20000 [00:00<09:24, 35.43it/s]

...

Number of samples loaded: 20000
Graphs shape: (20000, 2, 150, 150)
Features shape: (20000, 2, 150, 200)
Speedups (target) shape: (20000, 1)
Number of samples with equal graphs: 0
Number of features assigned (FA): 4709769
Invalid number of representations assigned: 2292509 (in relation to FA: 0.4867561445157926%)
Data saved to ./data/cdfgs_data_20000samples_150x150.npz


Samples generated:   0%|          | 4/20000 [00:00<09:21, 35.63it/s]

Samples information saved to ./data/selected_cdfgs_data_20000samples_150x150.yaml
Finished generation

Generating representation for: ('call',)


Samples generated: 100%|██████████| 20000/20000 [07:00<00:00, 47.55it/s]


Number of samples loaded: 20000
Graphs shape: (20000, 2, 150, 150)
Features shape: (20000, 2, 150, 200)
Speedups (target) shape: (20000, 1)
Number of samples with equal graphs: 1650
Number of features assigned (FA): 4709769
Invalid number of representations assigned: 534624 (in relation to FA: 0.11351384749443126%)
Data saved to ./data/cdfgs_call_20000samples_150x150.npz
Samples information saved to ./data/selected_cdfgs_call_20000samples_150x150.yaml
Finished generation



# Generate representations for (300x300) shape

### Common variables for all 300x300 representations

In [9]:
# Defining some useful variables
network_graph_shape = (300, 300)
network_features_shape = (300, 200)
n_samples = 20000  # Number of samples to generate. Each sample is composed by 2 graphs

cdfg_files = filter_cdfgs_files_below_shape(metadata_info, network_graph_shape)
samples = sample_cdfgs(cdfg_files, n_samples)

Number of graphs with adjacency matrix below shape (300, 300): 2910
Sampled 20000 CDFGs
First sample of 20000 samples (example of output): ('./data/ccpe-dados/cdfg.programl/programl.5000.02/extracted/Misc.mandel-2.0.21.progaml.yaml', './data/ccpe-dados/cdfg.programl/programl.5000.02/extracted/Misc.mandel-2.0.28.progaml.yaml', 1.011798022347439)


### Data Generation

In [10]:
representations_to_generate = [
    ('control', 'data', 'call'),
    ('control', 'data'),
    ('control', 'call'),
    ('data', 'call'),
    ('control',),
    ('data',),
    ('call',)
]

for representation in representations_to_generate:
    print(f'Generating representation for: {representation}')
    output_data_file, output_samples_file = get_output_filenames(data_dir, representation, n_samples, network_graph_shape)
    graphs, features, speedups = generate_samples(samples, network_graph_shape, network_features_shape, representation, embeddings_info)
    save_data_file(output_data_file, output_samples_file, samples, graphs, features, speedups)
    print('Finished generation\n')

Samples generated:   0%|          | 0/20000 [00:00<?, ?it/s]

Generating representation for: ('control', 'data', 'call')


Samples generated: 100%|██████████| 20000/20000 [14:18<00:00, 23.31it/s]


Number of samples loaded: 20000
Graphs shape: (20000, 2, 300, 300)
Features shape: (20000, 2, 300, 200)
Speedups (target) shape: (20000, 1)
Number of samples with equal graphs: 0
Number of features assigned (FA): 8557550
Invalid number of representations assigned: 5528569 (in relation to FA: 0.646045772446553%)
Data saved to ./data/cdfgs_control-data-call_20000samples_300x300.npz


Samples generated:   0%|          | 3/20000 [00:00<11:24, 29.21it/s]

...

Number of samples loaded: 20000
Graphs shape: (20000, 2, 300, 300)
Features shape: (20000, 2, 300, 200)
Speedups (target) shape: (20000, 1)
Number of samples with equal graphs: 0
Number of features assigned (FA): 8557550
Invalid number of representations assigned: 4291526 (in relation to FA: 0.5014900292723969%)
Data saved to ./data/cdfgs_data_20000samples_300x300.npz


Samples generated:   0%|          | 3/20000 [00:00<11:24, 29.21it/s]

Samples information saved to ./data/selected_cdfgs_data_20000samples_300x300.yaml
Finished generation

Generating representation for: ('call',)


Samples generated: 100%|██████████| 20000/20000 [13:48<00:00, 24.14it/s]


Number of samples loaded: 20000
Graphs shape: (20000, 2, 300, 300)
Features shape: (20000, 2, 300, 200)
Speedups (target) shape: (20000, 1)
Number of samples with equal graphs: 531
Number of features assigned (FA): 8557550
Invalid number of representations assigned: 607177 (in relation to FA: 0.07095220010400173%)
Data saved to ./data/cdfgs_call_20000samples_300x300.npz
Samples information saved to ./data/selected_cdfgs_call_20000samples_300x300.yaml
Finished generation

